# BoltzPay + LangChain: Pay for API Data with AI Agents

This notebook demonstrates how to use **langchain-boltzpay** to give your LangChain agents the ability to discover, check, and pay for API endpoints using the x402 and ACP protocols.

Your agent pays automatically with USDC on Base or Solana -- you just call `agent.invoke()`.

In [ ]:
!pip install langchain-boltzpay langchain-openai

## Setup

**Prerequisites:**
- **Node.js 20+** -- the CLI bridge calls `npx @boltzpay/cli` under the hood
- **Coinbase CDP keys** (only needed for `fetch`, not for `check`/`quote`/`discover`)

Set your environment variables:

```bash
export OPENAI_API_KEY="sk-..."

# Only needed for paid fetch:
export COINBASE_API_KEY_ID="your-key-id"
export COINBASE_API_KEY_SECRET="your-key-secret"
export COINBASE_WALLET_SECRET="your-wallet-secret"
```

In [ ]:
from langchain_boltzpay import (
    BoltzPayFetchTool,
    BoltzPayCheckTool,
    BoltzPayQuoteTool,
    BoltzPayDiscoverTool,
)

print("Tools imported successfully!")

## Step 1: Discover Available APIs

The `discover` tool lists all compatible paid API endpoints. No credentials needed.

In [ ]:
discover = BoltzPayDiscoverTool()
result = discover._run()
print(result)

## Step 2: Check an API's Price

Before paying, check what an endpoint costs. This is free -- no wallet needed.

In [ ]:
check = BoltzPayCheckTool()
result = check._run(url="https://invy.bot/api")
print(result)

## Step 3: Create an Agent with BoltzPay Tools

Now let's create a full LangChain agent that can discover APIs, check prices, and fetch paid data autonomously.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create tools
tools = [
    BoltzPayDiscoverTool(),
    BoltzPayCheckTool(),
    BoltzPayQuoteTool(),
    BoltzPayFetchTool(),
]

# Create the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a data researcher. Use BoltzPay tools to access paid APIs."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent
result = executor.invoke({
    "input": "Discover available APIs and check the price of invy.bot"
})
print(result["output"])

## Next Steps

- **Documentation:** [boltzpay.ai](https://boltzpay.ai)
- **GitHub:** [github.com/leventilo/boltzpay](https://github.com/leventilo/boltzpay)
- **Multi-protocol:** BoltzPay supports both x402 (crypto-native) and ACP (Stripe-based) payment protocols
- **MCP Alternative:** For CrewAI or other MCP-compatible frameworks, use `npx @boltzpay/mcp` directly

BoltzPay handles protocol detection, payment signing, and settlement automatically. Your agent just calls `fetch`.